In [5]:
%env CENSUS_API_KEY your-census-api-key-here

env: CENSUS_API_KEY=422bd8139ec28746163da006c428b340e2a94d07


In [82]:
import os
import requests
import pandas as pd
import folium
import geopandas as gpd
from functools import lru_cache
from shapely import wkt

API_KEY = os.environ['CENSUS_API_KEY']
STATE_CODE = '24'  # Maryland state code
COUNTY_CODE = '031'  # Montgomery County FIPS code

# ACS 5-year data profile
ACS_BASE_URL = 'https://api.census.gov/data/2019/acs/acs5/profile'

# Variable codes for the requested data
MEDIAN_INCOME = 'DP03_0062E'
MEAN_INCOME = 'DP03_0063E'
INC_10TH_PERCENTILE = 'DP03_0066E'
INC_90TH_PERCENTILE = 'DP03_0067E'

GEOJSON_FILE = 'Maryland_Census_Data_-_Census_Tracts.geojson'

def fetch_acs_data():
    url = f"{ACS_BASE_URL}?get=NAME,{MEDIAN_INCOME},{MEAN_INCOME},{INC_10TH_PERCENTILE},{INC_90TH_PERCENTILE}&for=tract:*&in=state:{STATE_CODE}&in=county:{COUNTY_CODE}&key={API_KEY}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return data

def process_acs_data(data):
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    df['TRACT'] = df['NAME'].apply(lambda x: x.split(',')[0].split(' ')[-1])
    df['TRACT'] = df['TRACT'].astype(str).apply(lambda x: x if '.' in x else x + '.00')
    df.rename(columns={
        'NAME': 'Census Tract',
        MEDIAN_INCOME: 'Median Income',
        MEAN_INCOME: 'Mean Income',
        INC_10TH_PERCENTILE: '10th Percentile Income',
        INC_90TH_PERCENTILE: '90th Percentile Income',
    }, inplace=True)
    
    df['90th Percentile Income'] = df['90th Percentile Income'].astype(int)
    df['10th Percentile Income'] = df['10th Percentile Income'].astype(int)
    df['Median Income'] = df['Median Income'].astype(int)
    return df


@lru_cache(maxsize=None)
def fetch_geojson(geojson_file):
    geo_data = gpd.read_file(geojson_file)
    geo_data['TRACT'] = geo_data['GEODESC']
    geo_data = geo_data[geo_data['CNTY2010'] == "24031"]
    geo_data.set_index("TRACT", inplace = True)
    return geo_data


def merge_geojson(geo_data, data):
    # Merge GeoJSON data with ACS data
#     merged_data = geo_data
    merged_data = geo_data.merge(data, left_on='GEODESC', right_on='TRACT')
    
    #display(merged_data.columns.tolist())
    
    #merged_data['geometry'] = merged_data['geometry'].astype(str)
    #merged_data.to_csv("output_file.csv", index=False)
    
    return merged_data

    
def create_map(acs_data, geo_data):
    m = folium.Map(location=[39.1547, -77.2405], zoom_start=10)


#     # Create choropleth map
#     folium.Choropleth(
#         geo_data=geo_data,
#         name='choropleth',
#         data=acs_data,
#         columns=['TRACT', '90th Percentile Income'],
#         key_on = 'feature.properties.GEODESC',
#         fill_color='YlOrRd',
#         fill_opacity=0.7,
#         line_opacity=0.2,
#         legend_name='90th Percentile Income',
#         smooth_factor=0
#     ).add_to(m)
#     folium.LayerControl().add_to(m)

    folium.Choropleth(
        geo_data = geo_data,
        name="choropleth",
        data=acs_data,
        columns=["TRACT", "Median Income"],
        key_on="feature.id",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="90th percentile income for MoCo by census tract",
    ).add_to(m)

# Add each census tract as a GeoJSON layer to the map
    merged_data = merge_geojson(geo_data, acs_data)
    
    folium.GeoJson(merged_data,
           name='Census Tracts',
           tooltip=folium.GeoJsonTooltip(fields=['Median Income'], aliases=['Median Income']),
           style_function=lambda x: {'fillColor': 'green', 'color': 'red', 'weight': 0.5},
           highlight_function=lambda x: {'weight': 3, 'color': 'black'},
           ).add_to(m)

    return m

def main():
    raw_data = fetch_acs_data()
    processed_data = process_acs_data(raw_data)
    geo_data = fetch_geojson(GEOJSON_FILE)
    
    map_ = create_map(processed_data, geo_data)
    map_.save('montgomery_county_income_map.html')

main()